# Text-to-SQL : Transform Sentences into SQL Queries
---
This notebook outlines the process of fine-tuning a transformer model to build a chatbot capable of translating natural language questions into SQL queries. 

We leverage the T5 transformer model, known for its flexibility across various NLP tasks, to train a robust and accurate text-to-SQL system.




In [9]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive



### 1. Setup and Library Imports

We start by importing the necessary libraries, including `transformers` for model and tokenizer setup, `torch` for handling training tasks, and `pandas` for data manipulation.


In [10]:
from transformers import T5ForConditionalGeneration, T5Tokenizer, Trainer, TrainingArguments,  Seq2SeqTrainingArguments
from datasets import Dataset
import torch
import json
import numpy as np
import pandas as pd

In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### 2. Model Selection and Initialization

We use the **T5-small Transformer** model, a versatile and popular model for text generation tasks.

T5 is well-suited for sequence-to-sequence tasks such as transforming natural language into SQL queries. 

We initialize the model and tokenizer for further processing and fine-tuning.

In [12]:
model_name = 't5-small'
model = T5ForConditionalGeneration.from_pretrained(model_name).to(device)
tokenizer = T5Tokenizer.from_pretrained(model_name)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


### 3. Data Loading and Preprocessing

The dataset comprises natural language questions paired with their respective SQL queries, stored in a Pandas DataFrame. Data preprocessing steps include:

- **Loading the Dataset**: The dataset is loaded with two primary columns:

  - **question**: containing natural language questions.
  - **answer**: containing the corresponding SQL queries.
- **Tokenization**: We use the T5 tokenizer to preprocess and encode the dataset, preparing it for input into the model. This includes tokenizing and creating separate sets for training, validation, and testing.

In [13]:
def load_dataset(file):
    with open(file) as file:
        content = json.load(file)
        df = pd.DataFrame(content)
        df = df[['question','answer']]
        dataset = Dataset.from_pandas(df)
        return dataset

In [14]:
def preprocess_data(examples):

    inputs = ["translate English to SQL: " + sentence for sentence in  examples['question'] ]
    targets = [ response for response in examples['answer']]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding='max_length')
    labels = tokenizer(targets, max_length=512, truncation=True, padding='max_length')
    model_inputs['labels'] = labels['input_ids']
    return model_inputs

In [15]:
train_dataset = load_dataset('./data/wikisql_train.json')
validation_dataset = load_dataset('./data/wikisql_validation.json')
test_dataset = load_dataset('./data/wikisql_test.json')

In [16]:
train_dataset

Dataset({
    features: ['question', 'answer'],
    num_rows: 56355
})

In [17]:
# Apply preprocessing to the dataset
tokenized_train_dataset = train_dataset.map(preprocess_data, batched=True, remove_columns=['question', 'answer'])
tokenized_validation_dataset = validation_dataset.map(preprocess_data, batched=True, remove_columns=['question', 'answer'])
tokenized_test_dataset = test_dataset.map(preprocess_data, batched=True, remove_columns=['question', 'answer'])


Map:   0%|          | 0/56355 [00:00<?, ? examples/s]

Map:   0%|          | 0/8421 [00:00<?, ? examples/s]

Map:   0%|          | 0/15878 [00:00<?, ? examples/s]


### 4. Model Fine-Tuning

We fine-tune the T5 model on the text-to-SQL task with the following setup:

- **Model Hyperparameters**: Defined arguments for optimal learning rates, batch sizes, and number of epochs.
- **Training**: The model is trained with a loss value reaching approximately **0.0105**, indicating a successful fit to the training data.
- **Evaluation**: The model's performance is evaluated on the validation set, achieving a loss value of **0.0217**.

In [18]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained("t5-small")

In [20]:
training_args = TrainingArguments(
    output_dir='/content/drive/MyDrive/results',
    evaluation_strategy='steps',
    learning_rate=2e-5,
    per_device_train_batch_size=10,
    per_device_eval_batch_size=10,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=1,
    save_steps=10_000,
    eval_steps=10_000,
    logging_steps=1_000,
    load_best_model_at_end=True,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [21]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_validation_dataset
)

In [22]:
trainer.train(resume_from_checkpoint=True)

There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight'].


Step,Training Loss,Validation Loss


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight'].


TrainOutput(global_step=16908, training_loss=0.010544284399107947, metrics={'train_runtime': 6082.5278, 'train_samples_per_second': 27.795, 'train_steps_per_second': 2.78, 'total_flos': 2.288156166586368e+16, 'train_loss': 0.010544284399107947, 'epoch': 3.0})

In [23]:
trainer.save_model("/content/drive/MyDrive/t5_text2sql")


### 5. Model Testing 

We test the fine-tuned T5 model on various natural language questions to evaluate its accuracy and SQL generation capabilities. 

In [24]:
eval_results = trainer.evaluate()
print(f"Evaluation results: {eval_results}")

Evaluation results: {'eval_loss': 0.021674759685993195, 'eval_runtime': 243.4455, 'eval_samples_per_second': 34.591, 'eval_steps_per_second': 3.463, 'epoch': 3.0}


### Testing on real-world examples to validate the model’s SQL query accuracy.

In [25]:
def generate_sql(query):
    input_text = "translate English to SQL: " + query
    input_ids = tokenizer.encode(input_text, return_tensors="pt").to(device)
    outputs = model.generate(input_ids, max_length=150, num_beams=4, early_stopping=True)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)


In [29]:
test_queries = [
    "How many employees are in the company?",
    "What is the average salary of employees?",
    "List all departments with more than 10 employees.",
    "How many student with grades better than 10?"
]

for query in test_queries:
    sql_query = generate_sql(query)
    print(f"Query: {query}\nGenerated SQL: {sql_query}\n")

Query: How many employees are in the company?
Generated SQL: SELECT COUNT Employees FROM table WHERE Company =

Query: What is the average salary of employees?
Generated SQL: SELECT AVG Salary FROM table WHERE Employees = employees

Query: List all departments with more than 10 employees.
Generated SQL: SELECT Department FROM table WHERE Employees > 10

Query: How many student with grades better than 10?
Generated SQL: SELECT COUNT Student FROM table WHERE Grades > 10



In [30]:
generate_sql("How many niggas are in Paris?")

'SELECT COUNT niggas FROM table WHERE Location = paris'

---

## Results and Observations

Through fine-tuning, the T5 model demonstrates effective translation of natural language questions into SQL queries. With further adjustments, it can be refined for practical deployment as a text-to-SQL chatbot.

- **HOUDA MOUDNI**
- **CHADI MOUNTASSIR**